https://www.opentopodata.org/api/

https://www.gpxz.io/blog/gpx-file-to-pandas

https://www.gpxz.io/blog/rayshading-in-python

In [9]:
import gpxpy
import pandas as pd
from numpy_ext import rolling_apply
import matplotlib.pyplot as plt
import geopy.distance
from numpy_ext import rolling_apply
import matplotlib.pyplot as plt



# Load gpx.
gpx_path = '../data/pineapple_pass.gpx'
with open(gpx_path) as f:
    gpx = gpxpy.parse(f)

# Convert to a dataframe one point at a time.
points = []
for segment in gpx.tracks[0].segments:
    for p in segment.points:
        points.append({
#             'time': p.time,
            'latitude': p.latitude,
            'longitude': p.longitude,
#             'elevation': p.elevation,
        })
df = pd.DataFrame.from_records(points)


In [5]:
df

,latitude,longitude
0,47.447957,-121.430683
1,47.450336,-121.436884
2,47.450873,-121.438386
3,47.452687,-121.442227
4,47.453209,-121.443064
...,...,...
218,47.451758,-121.453450
219,47.449843,-121.453042
220,47.448900,-121.453042
221,47.446694,-121.452227


In [10]:


# Cumulative distance.
coords = [(p.latitude, p.longitude) for p in df.itertuples()]
df['distance'] = [0] + [geopy.distance.distance(from_, to).m for from_, to in zip(coords[:-1], coords[1:])]
df['cumulative_distance'] = df.distance.cumsum()


# # Timing.
# df['duration'] = df.time.diff().dt.total_seconds().fillna(0)
# df['cumulative_duration'] = df.duration.cumsum()
# df['pace_metric'] = pd.Series((df.duration / 60) / (df.distance / 1000)).bfill()


In [12]:


# def rolling_metric_pace(duration, distance):
#     return (duration.sum() / 60) / (distance.sum() / 1000)


# x = df.cumulative_distance
# y = rolling_apply(rolling_metric_pace, 10, df.duration.values, df.distance.values)

# plt.plot(x, y)


In [13]:
import numpy as np
import requests
import pdb
BATCH_SIZE = 50  # 512 for paid customers.


def gpxz_elevation(lats, lons):
    elevations = []
    n_chunks = int(len(lats) // BATCH_SIZE)  + 1
    lat_chunks = np.array_split(lats, n_chunks) 
    lon_chunks = np.array_split(lons, n_chunks)
    for lat_chunk, lon_chunk in zip(lat_chunks, lon_chunks):
        latlons = '|'.join(f'{lat},{lon}' for lat, lon in zip(lat_chunk, lon_chunk))


        response = requests.post("https://api.opentopodata.org/v1/ned10m", data={"locations": latlons})

        response.raise_for_status()
        elevations += [r['elevation'] for r in response.json()['results']]
    return elevations

df['elevation'] = gpxz_elevation(df.latitude, df.longitude)




In [14]:
df

,latitude,longitude,distance,cumulative_distance,elevation
0,47.447957,-121.430683,0.000000,0.000000,987.140564
1,47.450336,-121.436884,537.329852,537.329852,1033.899170
2,47.450873,-121.438386,128.039444,665.369296,1034.375977
3,47.452687,-121.442227,352.933850,1018.303145,1039.133667
4,47.453209,-121.443064,85.762250,1104.065395,1047.876221
...,...,...,...,...,...
218,47.451758,-121.453450,0.000000,2370.146323,1354.191772
219,47.449843,-121.453042,215.155248,2585.301571,1418.149292
220,47.448900,-121.453042,104.863139,2690.164709,1416.988403
221,47.446694,-121.452227,252.818750,2942.983459,1491.460449
